In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('glassdoor data scientist salary.csv', na_values = '-1')

In [3]:
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist - Intermediate,NaN,KellyMitchell matches the best IT and business...,4.1,KellyMitchell\n4.1,"Saint Louis, MO",1001 to 5000 Employees,1998.0,Company - Private,Staffing & Outsourcing,Business Services,$50 to $100 million (USD)
1,Data Scientist,$87K - $142K (Glassdoor est.),"At Farm Credit Mid-America, we believe people ...",4.1,Farm Credit Mid-America\n4.1,"Louisville, KY",1001 to 5000 Employees,1985.0,Company - Private,Lending,Finance,$100 to $500 million (USD)
2,Research Scientist,NaN,Job Description:\nOur Reston office is seeking...,3.0,"Metron, Inc.\n3.0","Reston, VA",51 to 200 Employees,NaN,Company - Private,Electrical & Electronic Manufacturing,Manufacturing,$25 to $50 million (USD)
3,Junior Data Scientist,NaN,"Chubb is seeking a creative, innovative, and c...",NaN,WhiteHat,"New York, NY",NaN,NaN,NaN,NaN,NaN,NaN
4,Associate Clinical Data Scientist (Healthcare ...,$58K - $98K (Glassdoor est.),Job Summary\nThe Associate Clinical Data Scien...,2.6,RadNet\n2.6,"Cambridge, MA",5001 to 10000 Employees,1984.0,Company - Public,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          2500 non-null   object 
 1   Salary Estimate    1923 non-null   object 
 2   Job Description    2499 non-null   object 
 3   Rating             2272 non-null   float64
 4   Company Name       2500 non-null   object 
 5   Location           2500 non-null   object 
 6   Size               2178 non-null   object 
 7   Founded            1891 non-null   float64
 8   Type of ownership  2178 non-null   object 
 9   Industry           2065 non-null   object 
 10  Sector             2065 non-null   object 
 11  Revenue            2176 non-null   object 
dtypes: float64(2), object(10)
memory usage: 234.5+ KB


## Salary parsing

In [5]:
df.dropna(subset = ['Salary Estimate'], inplace = True)
df.reset_index(drop = True, inplace = True)
df.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,$87K - $142K (Glassdoor est.),"At Farm Credit Mid-America, we believe people ...",4.1,Farm Credit Mid-America\n4.1,"Louisville, KY",1001 to 5000 Employees,1985.0,Company - Private,Lending,Finance,$100 to $500 million (USD)
1,Associate Clinical Data Scientist (Healthcare ...,$58K - $98K (Glassdoor est.),Job Summary\nThe Associate Clinical Data Scien...,2.6,RadNet\n2.6,"Cambridge, MA",5001 to 10000 Employees,1984.0,Company - Public,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD)
2,FCC Data Scientist,$73K - $121K (Glassdoor est.),Summary: The FCC Analytics group is responsibl...,2.9,BankUnited\n2.9,"Miami Lakes, FL",1001 to 5000 Employees,2009.0,Company - Public,Banks & Credit Unions,Finance,$10+ billion (USD)
3,Data Analyst I,$42K - $74K (Glassdoor est.),Overview\n\n\nWestern National is seeking an e...,3.9,Western National Insurance\n3.9,"Edina, MN",501 to 1000 Employees,1901.0,Company - Private,Insurance Carriers,Insurance,$500 million to $1 billion (USD)
4,Data Analyst,$65K - $79K (Employer est.),Data Analyst\nCorporate Headquarters\n12575 Ul...,3.6,Uline\n3.6,"Pleasant Prairie, WI",5001 to 10000 Employees,1980.0,Company - Private,Wholesale,Business Services,Unknown / Non-Applicable


In [6]:
per_hour = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
employer = df['Salary Estimate'].apply(lambda x: 1 if 'employer' in x.lower() else 0)

df['Per Hour'] = per_hour
df['Employer Provided'] = employer

In [7]:
def salary_parse(salary):
    salary_split = salary.split('(')[0]
    if salary_split.endswith('r'):
        salary_split = salary.split(' Per')[0].split('-')
        salary_split = [int(i.replace('$', '')) for i in salary_split]
        salary_split = [i * 40 * 50 for i in salary_split]
    else:
        salary_split = salary.split(' (')[0].split(' - ')
        salary_split = [int(i.replace('$', '').replace('K', '')) for i in salary_split]
        salary_split = [i * 1000 for i in salary_split]
    return salary_split

In [10]:
df['Salary Lower'] = df['Salary Estimate'].apply(salary_parse).apply(lambda x: x[0])
df['Salary Higher'] = df['Salary Estimate'].apply(salary_parse).apply(lambda x: x[1])
df['Salary Average'] = ((df['Salary Lower'] + df['Salary Higher']) / 2).astype(int)
df[['Salary Estimate', 'Per Hour', 'Employer Provided', 'Salary Lower', 'Salary Higher', 'Salary Average']].head(10)

,Salary Estimate,Per Hour,Employer Provided,Salary Lower,Salary Higher,Salary Average
0,$87K - $142K (Glassdoor est.),0,0,87000,142000,114500
1,$58K - $98K (Glassdoor est.),0,0,58000,98000,78000
2,$73K - $121K (Glassdoor est.),0,0,73000,121000,97000
3,$42K - $74K (Glassdoor est.),0,0,42000,74000,58000
4,$65K - $79K (Employer est.),0,1,65000,79000,72000
5,$11K - $96K (Glassdoor est.),0,0,11000,96000,53500
6,$47K - $84K (Glassdoor est.),0,0,47000,84000,65500
7,$47K - $84K (Glassdoor est.),0,0,47000,84000,65500
8,$94K - $152K (Glassdoor est.),0,0,94000,152000,123000
9,$88K - $144K (Glassdoor est.),0,0,88000,144000,116000


## Company name text only

## State field

## Age of company

## Parsing of job description